In [ ]:
# forked from https://www.kaggle.com/ulrich07/ridge-regression-starter
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from nltk.corpus import stopwords


## LOADING

In [ ]:
ROOT_DIR = "../input/commonlitreadabilityprize"

## PROCESSING

In [ ]:
tr = pd.read_csv(f"{ROOT_DIR}/train.csv")
te = pd.read_csv(f"{ROOT_DIR}/test.csv")

In [ ]:
#vec = TfidfVectorizer(max_features=1000,stop_words = {'english'}) # V3 0.738
vec = TfidfVectorizer(max_df=0.8,min_df=0.2,max_features=100000,stop_words = {'english'}) # V4


X = vec.fit_transform(tr["excerpt"]).toarray()
Xe = vec.transform(te["excerpt"]).toarray()

In [ ]:
y = tr["target"].values

## TRAINING AND PREDICTION

In [ ]:
skf = KFold(n_splits=5)
folds = list(skf.split(X))

In [ ]:
oof = np.zeros(y.shape)
pred = np.zeros((Xe.shape[0],))
for idx in range(5):
    print("FOLD: ", idx + 1)
    tr_idx, val_idx = folds[idx]
    reg = Ridge()
    reg.fit(X[tr_idx], y[tr_idx])
    oof[val_idx] =reg.predict(X[val_idx])
    pred += reg.predict(Xe) / 5
#===============

In [ ]:
mse = mean_squared_error(y, oof, squared=False)
print("OOF MSE:",mse)

## SUBMISSION

In [ ]:
sub = te[["id"]].copy()
sub["target"] = pred

In [ ]:
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)